In [1]:
from courseRegistrationHelper import *

#login()

#time.sleep(20)

#driver.quit()

In [ ]:
from courseRegistrationHelper import *

def register(course_list:list, once:bool) -> None:
    try:
        while True:
            for course in course_list:
                course_elements = course.split("-")
                if is_group_available(course_elements[0], course_elements[1]):
                    #checkout_cart()
                    print("registered: " + course)
                    if once:
                        raise KeyboardInterrupt
                    else:
                        course_list.remove(course)
                        break
                else:
                    print("no room available: " + course)
    except KeyboardInterrupt:
        pass


#list = ["234124-11", "234124-13", "234124-15", "394804-18","394804-19", "324439-10"]
list = ["234124-11", "234124-13", "394804-18"]
print(list)
register(list, False)
print(list)

In [ ]:
from courseRegistrationHelper import *

SPANS_XPATH = "/html/body/div[2]/div[3]/div/div/section/div/div/div[3]/div/div[1]/div/span"
WebDriverWait(driver, TIMEOUT).until(EC.presence_of_element_located((By.XPATH, SPANS_XPATH)))
groups = driver.find_elements(By.XPATH, SPANS_XPATH)
if len(groups) == 0:
    SPANS_XPATH = "/html/body/div[2]/div[3]/div/div/section/div/div/div[2]/div/div[1]/div/span"
    groups = driver.find_elements(By.XPATH, SPANS_XPATH)

openings = [None] * len(groups)
for index, group in enumerate(groups):
    if driver.find_element(By.XPATH, SPANS_XPATH + "[" + str(index+1) + "]/div/table/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/div[2]").get_attribute("class") == IS_AVAILABLE:
        openings[index] = True
    else:
        openings[index] = False
print(openings)
print(openings.count(True))